In [1]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from tqdm import tqdm

# Dataset Class
class XRayDataset(Dataset):
    def __init__(self, csv_file, img_folder, transform=None):
        self.csv = pd.read_csv(csv_file)
        self.img_folder = img_folder
        self.transform = transform
        self.pairs = self.csv.groupby('uid')  
        self.valid_uids = self.check_integrity()

    def check_integrity(self):
        valid_uids = []
        for uid, group in self.pairs:
            frontal_data = group[group['projection'] == 'Frontal']
            lateral_data = group[group['projection'] == 'Lateral']
            if frontal_data.empty or lateral_data.empty:
                continue
            valid_uids.append(uid)
        return valid_uids

    def __len__(self):
        return len(self.valid_uids)

    def __getitem__(self, idx):
        group_key = self.valid_uids[idx]
        uid = self.pairs.get_group(group_key)

        frontal_img_path = os.path.join(self.img_folder, uid[uid['projection'] == 'Frontal']['filename'].values[0])
        lateral_img_path = os.path.join(self.img_folder, uid[uid['projection'] == 'Lateral']['filename'].values[0])
        frontal_img = Image.open(frontal_img_path).convert("RGB")
        lateral_img = Image.open(lateral_img_path).convert("RGB")

        if self.transform:
            frontal_img = self.transform(frontal_img)
            lateral_img = self.transform(lateral_img)

        return frontal_img, lateral_img

# Define the Dense Block
class DenseBlock(nn.Module):
    def __init__(self, in_channels, growth_rate, num_layers):
        super(DenseBlock, self).__init__()
        self.layers = nn.ModuleList()
        for i in range(num_layers):
            self.layers.append(nn.Sequential(
                nn.Conv2d(in_channels + i * growth_rate, growth_rate, kernel_size=3, padding=1),
                nn.InstanceNorm2d(growth_rate),
                nn.ReLU(inplace=True)
            ))

    def forward(self, x):
        for layer in self.layers:
            new_features = layer(x)
            x = torch.cat([x, new_features], dim=1)
        return x

# Define the Basic3D Block
class Basic3D(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Basic3D, self).__init__()
        self.block = nn.Sequential(
            nn.Conv3d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.InstanceNorm3d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.block(x)

# Define 2D to 3D Connection (Connection-C)
class Connection2Dto3D(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Connection2Dto3D, self).__init__()
        self.conv2d = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv3d = nn.Conv3d(out_channels, out_channels, kernel_size=3, padding=1)

    def forward(self, x):
        x = self.conv2d(x)
        x = x.unsqueeze(2)
        x = self.conv3d(x)
        return x

# Define Generator Architecture
class Generator(nn.Module):
    def __init__(self, in_channels, growth_rate, num_dense_layers, out_channels, image_size):
        super(Generator, self).__init__()

        self.num_upconv_layers = int(torch.log2(torch.tensor(image_size // 4)).item())

        self.encoder1 = nn.Sequential(
            nn.Conv2d(in_channels, growth_rate, kernel_size=3, padding=1),
            DenseBlock(growth_rate, growth_rate, num_dense_layers),
            nn.Conv2d(growth_rate * (num_dense_layers + 1), growth_rate, kernel_size=3, stride=2, padding=1)  # Compression
        )

        self.encoder2 = nn.Sequential(
            nn.Conv2d(in_channels, growth_rate, kernel_size=3, padding=1),
            DenseBlock(growth_rate, growth_rate, num_dense_layers),
            nn.Conv2d(growth_rate * (num_dense_layers + 1), growth_rate, kernel_size=3, stride=2, padding=1)  # Compression
        )

        self.connection_a = Connection2Dto3D(growth_rate, growth_rate)
        self.connection_b = nn.Conv3d(growth_rate * 2, growth_rate, kernel_size=3, padding=1)
        self.connection_c = nn.Conv3d(growth_rate, growth_rate, kernel_size=3, padding=1)

        self.upconv_layers = nn.ModuleList()
        for _ in range(self.num_upconv_layers):
            self.upconv_layers.append(nn.Sequential(
                nn.ConvTranspose3d(growth_rate, growth_rate, kernel_size=4, stride=2, padding=1),
                nn.InstanceNorm3d(growth_rate),
                nn.ReLU(inplace=True)
            ))

        self.final_layer = nn.Conv3d(growth_rate, out_channels, kernel_size=3, padding=1)

    def forward(self, x1, x2):

        x1 = self.encoder1(x1)
        x2 = self.encoder2(x2)
        x1 = self.connection_a(x1)
        x2 = self.connection_a(x2)
        x = torch.cat([x1, x2], dim=1)
        x = self.connection_b(x)
        for upconv in self.upconv_layers:
            x = upconv(x)

        x = self.final_layer(x)
        return x

# Training logic
if __name__ == "__main__":

    IMAGE_FOLDER = '/projectnb/ec523kb/projects/teams_Fall_2024/Team_11/Adwait/Work_on_this_code/images/images_normalized/'
    CSV_FILE = '/projectnb/ec523kb/projects/teams_Fall_2024/Team_11/Adwait/Work_on_this_code/indiana_projections.csv'
    BATCH_SIZE = 32
    EPOCHS = 10
    IMAGE_SIZE = 100
    LATENT_DIM = 1000
    IN_CHANNELS = 3
    OUT_CHANNELS = 1
    GROWTH_RATE = 16
    NUM_DENSE_LAYERS = 4

    transform = transforms.Compose([
        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        transforms.ToTensor()
    ])


    dataset = XRayDataset(CSV_FILE, IMAGE_FOLDER, transform)
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    generator = Generator(
        in_channels=IN_CHANNELS,
        growth_rate=GROWTH_RATE,
        num_dense_layers=NUM_DENSE_LAYERS,
        out_channels=OUT_CHANNELS,
        image_size=IMAGE_SIZE
    )
    generator = nn.DataParallel(generator)  # Enable multi-GPU
    generator = generator.to(device)

    optimizer = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    criterion = nn.MSELoss()

    for epoch in range(EPOCHS):
        generator.train()
        epoch_loss = 0

        with tqdm(dataloader, unit="batch") as tepoch:
            for frontal_img, lateral_img in tepoch:
                tepoch.set_description(f"Epoch {epoch + 1}")

                frontal_img = frontal_img.to(device)
                lateral_img = lateral_img.to(device)


                optimizer.zero_grad()

                output = generator(frontal_img, lateral_img)
                loss = criterion(output, torch.zeros_like(output).to(device))

                loss.backward()
                optimizer.step()

                epoch_loss += loss.item()
                tepoch.set_postfix(loss=loss.item())

        print(f"Epoch {epoch + 1}, Loss: {epoch_loss:.4f}")
    print("Training finished.")
    torch.save(generator.state_dict(), f"generator_{BATCH_SIZE}_{IMAGE_SIZE}_epoch_{EPOCHS}.pth")
#     torch.save(discriminator.state_dict(), f"discriminator_{BATCH_SIZE}_{IMAGE_SIZE}_epoch_{EPOCHS}.pth")

    print(f"Models saved at epoch {EPOCHS} with batch size {BATCH_SIZE} and image size {IMAGE_SIZE}")


  0%|          | 0/106 [00:00<?, ?batch/s]

Epoch 1, Loss: 0.9886


  0%|          | 0/106 [00:00<?, ?batch/s]

Epoch 2, Loss: 0.0410


  0%|          | 0/106 [00:00<?, ?batch/s]

...

  0%|          | 0/106 [00:00<?, ?batch/s]

Epoch 9, Loss: 0.0108


Epoch 10: 100%|██████████| 106/106 [07:32<00:00,  4.27s/batch, loss=0.000121]


Epoch 10, Loss: 0.0110
Training finished.


NameError: name 'discriminator' is not defined

In [ ]:
# import os
# import pandas as pd
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader
# from torchvision import transforms
# from PIL import Image
# from tqdm import tqdm

# # Dataset Class
# class XRayDataset(Dataset):
#     def __init__(self, csv_file, img_folder, transform=None):
#         self.csv = pd.read_csv(csv_file)
#         self.img_folder = img_folder
#         self.transform = transform
#         self.pairs = self.csv.groupby('uid')  # Group by UID
#         self.valid_uids = self.check_integrity()  # Ensure dataset integrity

#     def check_integrity(self):
#         valid_uids = []
#         for uid, group in self.pairs:
#             frontal_data = group[group['projection'] == 'Frontal']
#             lateral_data = group[group['projection'] == 'Lateral']
#             if frontal_data.empty or lateral_data.empty:
#                 continue
#             valid_uids.append(uid)
#         return valid_uids

#     def __len__(self):
#         return len(self.valid_uids)

#     def __getitem__(self, idx):
#         group_key = self.valid_uids[idx]
#         uid = self.pairs.get_group(group_key)

#         # Find the paths for the frontal and lateral images
#         frontal_img_path = os.path.join(self.img_folder, uid[uid['projection'] == 'Frontal']['filename'].values[0])
#         lateral_img_path = os.path.join(self.img_folder, uid[uid['projection'] == 'Lateral']['filename'].values[0])

#         # Load images
#         frontal_img = Image.open(frontal_img_path).convert("RGB")
#         lateral_img = Image.open(lateral_img_path).convert("RGB")

#         if self.transform:
#             frontal_img = self.transform(frontal_img)
#             lateral_img = self.transform(lateral_img)

#         return frontal_img, lateral_img

# # Define the Dense Block
# class DenseBlock(nn.Module):
#     def __init__(self, in_channels, growth_rate, num_layers):
#         super(DenseBlock, self).__init__()
#         self.layers = nn.ModuleList()
#         for i in range(num_layers):
#             self.layers.append(nn.Sequential(
#                 nn.Conv2d(in_channels + i * growth_rate, growth_rate, kernel_size=3, padding=1),
#                 nn.InstanceNorm2d(growth_rate),
#                 nn.ReLU(inplace=True)
#             ))

#     def forward(self, x):
#         for layer in self.layers:
#             new_features = layer(x)
#             x = torch.cat([x, new_features], dim=1)
#         return x

# # Define the Basic3D Block
# class Basic3D(nn.Module):
#     def __init__(self, in_channels, out_channels):
#         super(Basic3D, self).__init__()
#         self.block = nn.Sequential(
#             nn.Conv3d(in_channels, out_channels, kernel_size=3, padding=1),
#             nn.InstanceNorm3d(out_channels),
#             nn.ReLU(inplace=True)
#         )

#     def forward(self, x):
#         return self.block(x)

# # Define 2D to 3D Connection (Connection-C)
# class Connection2Dto3D(nn.Module):
#     def __init__(self, in_channels, out_channels):
#         super(Connection2Dto3D, self).__init__()
#         self.conv2d = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
#         self.conv3d = nn.Conv3d(out_channels, out_channels, kernel_size=3, padding=1)

#     def forward(self, x):
#         x = self.conv2d(x)
#         x = x.unsqueeze(2)  # Add depth dimension
#         x = self.conv3d(x)
#         return x

# # Define Generator Architecture
# class Generator(nn.Module):
#     def __init__(self, in_channels, growth_rate, num_dense_layers, out_channels, image_size):
#         super(Generator, self).__init__()

#         # Calculate depth of up-convolutions based on image size
#         self.num_upconv_layers = int(torch.log2(torch.tensor(image_size // 4)).item())

#         # Encoder: 2D Branches
#         self.encoder1 = nn.Sequential(
#             nn.Conv2d(in_channels, growth_rate, kernel_size=3, padding=1),
#             DenseBlock(growth_rate, growth_rate, num_dense_layers),
#             nn.Conv2d(growth_rate * (num_dense_layers + 1), growth_rate, kernel_size=3, stride=2, padding=1)  # Compression
#         )

#         self.encoder2 = nn.Sequential(
#             nn.Conv2d(in_channels, growth_rate, kernel_size=3, padding=1),
#             DenseBlock(growth_rate, growth_rate, num_dense_layers),
#             nn.Conv2d(growth_rate * (num_dense_layers + 1), growth_rate, kernel_size=3, stride=2, padding=1)  # Compression
#         )

#         # Connection Layers
#         self.connection_a = Connection2Dto3D(growth_rate, growth_rate)
#         self.connection_b = nn.Conv3d(growth_rate * 2, growth_rate, kernel_size=3, padding=1)
#         self.connection_c = nn.Conv3d(growth_rate, growth_rate, kernel_size=3, padding=1)

#         # Decoder: 3D Branch
#         self.upconv_layers = nn.ModuleList()
#         for _ in range(self.num_upconv_layers):  # Dynamically add up-conv layers
#             self.upconv_layers.append(nn.Sequential(
#                 nn.ConvTranspose3d(growth_rate, growth_rate, kernel_size=4, stride=2, padding=1),
#                 nn.InstanceNorm3d(growth_rate),
#                 nn.ReLU(inplace=True)
#             ))

#         self.final_layer = nn.Conv3d(growth_rate, out_channels, kernel_size=3, padding=1)

#     def forward(self, x1, x2):
#         # Encode both inputs
#         x1 = self.encoder1(x1)
#         x2 = self.encoder2(x2)

#         # Combine with Connection-A
#         x1 = self.connection_a(x1)
#         x2 = self.connection_a(x2)
#         x = torch.cat([x1, x2], dim=1)  # Connection-B

#         # Pass through Connection-B
#         x = self.connection_b(x)

#         # Pass through up-convolutions (Decoder)
#         for upconv in self.upconv_layers:
#             x = upconv(x)

#         # Final 3D output
#         x = self.final_layer(x)
#         return x
    
    
# class Discriminator(nn.Module):
#     def __init__(self, in_channels, growth_rate, num_dense_layers):
#         super(Discriminator, self).__init__()
#         self.initial_conv = nn.Sequential(
#             nn.Conv2d(in_channels, growth_rate, kernel_size=3, padding=1),
#             nn.LeakyReLU(0.2, inplace=True)
#         )
#         self.dense_block = DenseBlock(growth_rate, growth_rate, num_dense_layers)
#         self.final_conv = nn.Sequential(
#             nn.Conv2d(growth_rate * (num_dense_layers + 1), 1, kernel_size=3, padding=1),
#             nn.Sigmoid()
#         )

#     def forward(self, x):
#         x = self.initial_conv(x)
#         x = self.dense_block(x)
#         x = self.final_conv(x)
#         return x


# # Training logic
# if __name__ == "__main__":

#     IMAGE_FOLDER = '/projectnb/ec523kb/projects/teams_Fall_2024/Team_11/Adwait/Work_on_this_code/images/images_normalized/'
#     CSV_FILE = '/projectnb/ec523kb/projects/teams_Fall_2024/Team_11/Adwait/Work_on_this_code/indiana_projections.csv'
#     BATCH_SIZE = 16
#     EPOCHS = 10
#     IMAGE_SIZE = 64
#     LATENT_DIM = 100
#     IN_CHANNELS = 3
#     OUT_CHANNELS = 1
#     GROWTH_RATE = 16
#     NUM_DENSE_LAYERS = 4

#     # Define transforms
#     transform = transforms.Compose([
#         transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
#         transforms.ToTensor()
#     ])

#     # Dataset and Dataloader
#     dataset = XRayDataset(CSV_FILE, IMAGE_FOLDER, transform)
#     dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     generator = Generator(
#         in_channels=IN_CHANNELS,
#         growth_rate=GROWTH_RATE,
#         num_dense_layers=NUM_DENSE_LAYERS,
#         out_channels=OUT_CHANNELS,
#         image_size=IMAGE_SIZE
#     )
#     generator = nn.DataParallel(generator)  # Enable multi-GPU
#     generator = generator.to(device)

#     adversarial_loss = nn.BCELoss()

#     discriminator = Discriminator(
#         in_channels=IN_CHANNELS,
#         growth_rate=GROWTH_RATE,
#         num_dense_layers=NUM_DENSE_LAYERS
#     ).to(device)
#     discriminator = nn.DataParallel(discriminator)  # Multi-GPU

#     optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
#     optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))

#     for epoch in range(EPOCHS):
#         generator.train()
#         discriminator.train()
#         g_loss_epoch = 0
#         d_loss_epoch = 0

#         with tqdm(dataloader, unit="batch") as tepoch:
#             for frontal_img, lateral_img in tepoch:
#                 tepoch.set_description(f"Epoch {epoch + 1}")

#                 frontal_img = frontal_img.to(device)
#                 lateral_img = lateral_img.to(device)

#                 # Generate fake 2D slices
#                 fake_slices = generator(frontal_img, lateral_img)

#                 # Prepare real 2D slices (placeholder target for now)
#                 real_slices = torch.zeros_like(fake_slices).to(device)  # Replace with real target

#                 # Adversarial ground truths
#                 real_labels = torch.ones(fake_slices.size(0), 1, 1, 1).to(device)
#                 fake_labels = torch.zeros(fake_slices.size(0), 1, 1, 1).to(device)

#                 # ---------------------
#                 #  Train Discriminator
#                 # ---------------------
#                 optimizer_D.zero_grad()

#                 # Real loss
#                 real_loss = adversarial_loss(discriminator(real_slices), real_labels)

#                 # Fake loss
#                 fake_loss = adversarial_loss(discriminator(fake_slices.detach()), fake_labels)

#                 # Total loss
#                 d_loss = (real_loss + fake_loss) / 2
#                 d_loss.backward()
#                 optimizer_D.step()

#                 # -----------------
#                 #  Train Generator
#                 # -----------------
#                 optimizer_G.zero_grad()

#                 # Generator loss (adversarial loss)
#                 g_loss = adversarial_loss(discriminator(fake_slices), real_labels)
#                 g_loss.backward()
#                 optimizer_G.step()

#                 # Log losses
#                 d_loss_epoch += d_loss.item()
#                 g_loss_epoch += g_loss.item()
#                 tepoch.set_postfix(g_loss=g_loss.item(), d_loss=d_loss.item())

#         print(f"Epoch {epoch + 1}, G Loss: {g_loss_epoch:.4f}, D Loss: {d_loss_epoch:.4f}")

